# Lab in Data Science: Final Project

Pierre Fouche, Matthias Leroy and Raphaël Steinmann

## Imports

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [2]:
import getpass
import pyspark
from datetime import datetime
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
from pyspark.sql.types import BooleanType
from pyspark.sql.window import Window
import math
import helpers
import pickle

%load_ext autoreload
%autoreload 2

## Initialize the `SparkSession`

In [3]:
conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '6g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=project-fouche>

In [4]:
# init spark session
spark = SparkSession(sc)

## Loading the data

## Data Processing

### Cleaning metadata
First, let's clean the metadata dataframe:

In [5]:
# load metadata
raw_metadata = spark.read.load('/datasets/project/metadata', format='com.databricks.spark.csv', header='false', sep='\\t')

In [6]:
# remove multiple spaces
metadata = raw_metadata.withColumn('_c0', functions.regexp_replace(raw_metadata._c0, '\s+', ' '))
# split into columns
metadata = metadata.withColumn('name', functions.split(metadata._c0, '%')[1])
for (name, index, type_) in [('station_ID',0, 'int'), ('long',1, 'double'), ('lat',2, 'double'), ('height',3, 'int')]:
    metadata = metadata.withColumn(name, functions.split(metadata._c0, ' ')[index].cast(type_))
# remove useless column
metadata = metadata.drop('_c0')
# trim name column to remove left/right blank
metadata = metadata.withColumn('name', functions.trim(metadata.name))

In [7]:
metadata.show(5)

+----------------+----------+---------+---------+------+
|            name|station_ID|     long|      lat|height|
+----------------+----------+---------+---------+------+
|       Bucuresti|         2|26.074412| 44.44677|     0|
|          Calais|         3| 1.811446|50.901549|     0|
|      Canterbury|         4| 1.075329|51.284212|     0|
|          Exeter|         5|-3.543547|50.729172|     0|
|Fideris, Bahnhof|         7| 9.733756|46.922368|   744|
+----------------+----------+---------+---------+------+
only showing top 5 rows



We will use the SBB data limited around the Zurich area. We will focus on all the stops within 10km of the Zurich train station. Let's get rid of all the stations that are too far away from Zurich:

In [8]:
metadata.count()

25935

In [9]:
# coordinates of Zürich main train station
lat_zurich = 47.3782
long_zurich = 8.5402

In [10]:
# convert to pandas dataframe
pandas_df = metadata.toPandas()
# keep only the stops that are located < 10km from Zurich HB
pandas_df['distance_to_zh'] = pandas_df.apply(lambda x: helpers.distance(x['long'], x['lat'], long_zurich, lat_zurich), axis=1)
pandas_df = pandas_df[pandas_df['distance_to_zh'] < 10]

In [11]:
# pandas_df.distance_to_zh.max()

In [12]:
# recreate spark dataframe from pandas dataframe
metadata = spark.createDataFrame(pandas_df)
# create dict of stations from pandas dataframe
stations = pandas_df.set_index('station_ID').to_dict('index')

### Cleaning main dataset

In [13]:
# load full data
# raw_df = spark.read.load('/datasets/project/istdaten/*/*', format='csv', header='true', inferSchema='true', sep=';')
# load sample data
raw_df = spark.read.load('/datasets/project/istdaten/2018/01', format='csv', header='true', inferSchema='true', sep=';')

In [14]:
# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'trip_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'BPUIC':'stop_id',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}

df = raw_df.selectExpr([k + ' as ' + fields[k] for k in fields])

In [15]:
# refactor dates
df = df.withColumn('date', functions.from_unixtime(functions.unix_timestamp('date', 'dd.MM.yyyy')))
df = df.withColumn('schedule_arrival', functions.from_unixtime(functions.unix_timestamp('schedule_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_arrival', functions.from_unixtime(functions.unix_timestamp('real_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('schedule_dep', functions.from_unixtime(functions.unix_timestamp('schedule_dep', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_dep', functions.from_unixtime(functions.unix_timestamp('real_dep', 'dd.MM.yyyy HH:mm')))

In [16]:
# add a column containing the weekday (monday=1, sunday=6)
df = df.withColumn('weekday', helpers.get_weekday(df.date))

In [17]:
# keep only the rows with stops near zurich
df = df.where(df.stop_id.isin([int(x) for x in list(pandas_df.station_ID.unique())]))

In [18]:
# there is still 51'571'541 rows in zurich area
# df.count()

In [19]:
# keep only date after the 10th of december, because the schedule changed
df = df.where(df.date > '10.12.2017 00:00')

In [20]:
# discard the rows when there is no stop here
df2 = df.where(df.no_stop_here == 'false')

## Modeling the network

### From stops to trips

In [21]:
# create a column with the schedule time that will be used to build the network
df2 = df2.withColumn('schedule_time', helpers.date_choice(df2.schedule_arrival, df2.schedule_dep))
#df2 = df2.withColumn('schedule_time', functions.from_unixtime(functions.unix_timestamp('schedule_time', 'dd.MM.yyyy HH:mm')))

# create a column that tells if a stop is the first/last one of its trip or in the middle
df2 = df2.withColumn('stop_type', helpers.stop_type(df2.schedule_dep, df2.schedule_arrival))

In [37]:
trips = df2.select(['trip_id', 'date', 'schedule_time', 'stop_id', 'stop_type', 'schedule_arrival', 'schedule_dep', 'transport_type', 'train_type', 'arr_forecast_status', 'weekday', 'real_arrival']).orderBy(['trip_id', 'schedule_time'], ascending=[0,0])

In [38]:
# duplicate the dataframe, shift the copy of one row and append it to the original
# this way, we have for each row the current stop and the next stop
w = Window().partitionBy(functions.col('trip_id')).orderBy(functions.col('trip_id'))
trips2 = trips.select("*", functions.lag("trip_id").over(w).alias("next_tid"))
trips2 = trips2.select("*", functions.lag("schedule_time").over(w).alias("next_time"))
trips2 = trips2.select("*", functions.lag("stop_id").over(w).alias("next_sid"))
trips2 = trips2.select("*", functions.lag("stop_type").over(w).alias("next_type"))
trips2 = trips2.select("*", functions.lag("schedule_arrival").over(w).alias("next_sched_arr"))
trips2 = trips2.select("*", functions.lag("schedule_dep").over(w).alias("next_sched_dep"))
trips2 = trips2.select("*", functions.lag("arr_forecast_status").over(w).alias("next_arr_forecast_status"))
trips2 = trips2.select("*", functions.lag("real_arrival").over(w).alias("next_real_arrival"))

trips2 = trips2.where(trips2.next_time.isNotNull())

In [39]:
#trips2.where(trips2.stop_type=='first').count()
#trips2.where(trips2.stop_type=='last').count()
#trips2.where(trips2.stop_type=='mid').count()

In [40]:
# create a new column telling if the edge is valid or not
# (i.e. if the stop and next stop are really part of the same ride)
trips3 = trips2.withColumn('is_valid', helpers.edge_is_valid(trips2.trip_id, trips2.schedule_time, trips2.stop_id, trips2.stop_type, trips2.next_tid, trips2.next_time, trips2.next_sid, trips2.next_type))

In [41]:
# keep only valid edges
trips4 = trips3.filter(trips3.is_valid=='true')

In [42]:
# trips4.select('stop_id', 'next_sid').distinct().count()
# gives 6606

# trips3.select('stop_id', 'next_sid').distinct().count()
# gives 8557

### For each day of the week, model the network
Get the edges of the network and the departure/arrival times for each trip (edge=trip)
We assume the schedule repeat every week, and we generate one schedule per weekday.
Days off have the same schedules as sundays.

In [ ]:
# creating a model for each day of the week
# this code needs to be run only once
typical_monday = '2018-01-15 00:00:00'
typical_tuesday = '2018-01-16 00:00:00'
typical_wednesday = '2018-01-17 00:00:00'
typical_thursday = '2018-01-18 00:00:00'
typical_friday = '2018-01-19 00:00:00'
typical_saturday = '2018-01-20 00:00:00'
typical_sunday = '2018-01-21 00:00:00'
typical_week = [typical_monday,typical_tuesday,typical_wednesday,typical_thursday,typical_friday,typical_saturday,typical_sunday]

In [ ]:
regenerate_models = False
models = []
days_names = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

# generate one network for each weekday and store them in pickles
if regenerate_models:
    for (date, day_name) in zip(typical_week, days_names):
        network = helpers.model_network(trips4, date)
        with open('./data/'+day_name+'.pickle', 'wb') as handle:
            pickle.dump(network, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(str(day_name) + ' done')

In [ ]:
# load the networks from the pickles
for day in days_names:
    with open('./data/'+ day +'.pickle', 'rb') as handle:
        network = pickle.load(handle)
    models.append(network)

In [ ]:
models[0][list(models[0].keys())[0]]

## Prediction / Regression

Deux idées : Le but c'est de prévoir le retard ou l'avance d'un départ ou d'une arrivée pour un arrêt précis à un moment précis et un trip précis.

- Faire une vraie regression ou n'importe quel algo pour déterminer le retard. On utilise comme features : l'arrêt (catégorie), latitude longitude (peut être mettre ensemble en tuple), jour precis ou jour de la semaine ?, le type de transport (bus-train ...), le trip id ou redondant ???, 

- Faire une moyenne, à un arrêt, un trip, un jour, une heure la moyenne de retard qu'il a eu dans ses conditions ... 

On va avoir besoin des trips id dans le dico histoire de pouvoir lier le graph des arrêts avec le réél trajet, quel bus/train fait cet edge.

### Preprocessing :

In [43]:
trips5 = trips4.drop('stop_type', 'next_type', 'is_valid', 'arr_forcast_status', 
                     'schedule_time', 'date', 'schedule_arrival', 'next_sched_dep', 
                     'next_time', 'next_tid', 'real_arrival', 'arr_forecast_status')

In [45]:
@functions.udf
def create_interval(date):
    # rush_hour = 6/9 17/19 = 0
    # not_rush_hour = 0/6 9/17 19/24  = 1
    date = date.split(' ')[1]
    if (date >= '06:00:00' and date <= '09:00:00') or (date >= '17:00:00' and date <= '19:00:00'):
        return 0
    else:
        return 1

In [48]:
# We compute the arrival delay of each stop station
df3 = trips5.withColumn("delay_arrival", 
                     functions.unix_timestamp('next_real_arrival', 'yyyy-MM-dd HH:mm:ss') -
                     functions.unix_timestamp('next_sched_arr', 'yyyy-MM-dd HH:mm:ss'))

# We create time interval in order to differentiate rush hour and other time of the day
# We have to try different combinations of interval time (that gave best prediction)
df3 = df3.withColumn('arrival_interval', create_interval(df3.next_sched_arr))

In [50]:
# We create 2 dataframes, one for the arrival (without the first departure stations)
arrival_df = df3.filter(df3.next_arr_forecast_status == "GESCHAETZT")
# only time of train (Zug) was evaluated
arrival_df = arrival_df.drop('next_arr_forecast_status', 'transport_type')

In [52]:
arrival_df.count()

309380

In [53]:
df3.count()

6685965

In [56]:
df3.groupby('weekday').count().show()

+-------+-------+
|weekday|  count|
+-------+-------+
|      3| 937194|
|      0|1094625|
|      5| 810546|
|      6| 626375|
|      1|1092998|
|      4| 950304|
|      2|1173894|
+-------+-------+



In [57]:
arrival_df.groupby('weekday').count().show()

+-------+-----+
|weekday|count|
+-------+-----+
|      3|40439|
|      0|50615|
|      5|38970|
|      6|38877|
|      1|49455|
|      4|40572|
|      2|50451|
+-------+-----+



In [60]:
arrival_pd_df = arrival_df.toPandas()

In [61]:
arrival_pd_df.head()

,trip_id,stop_id,schedule_dep,transport_type,train_type,weekday,next_sid,next_sched_arr,next_real_arrival,delay_arrival,arrival_interval
0,85:11:13752:001,8502221,2018-01-28 02:12:00,Zug,SN,6,8502222,2018-01-28 02:16:00,2018-01-28 02:18:00,120,1
1,85:11:13752:001,8502229,2018-01-28 02:07:00,Zug,SN,6,8502221,2018-01-28 02:12:00,2018-01-28 02:13:00,60,1
2,85:11:13752:001,8502220,2018-01-28 02:06:00,Zug,SN,6,8502229,2018-01-28 02:07:00,2018-01-28 02:10:00,180,1
3,85:11:13752:001,8503001,2018-01-28 02:01:00,Zug,SN,6,8502220,2018-01-28 02:06:00,2018-01-28 02:08:00,120,1
4,85:11:13752:001,8503020,2018-01-28 01:59:00,Zug,SN,6,8503001,2018-01-28 02:01:00,2018-01-28 02:02:00,60,1


In [63]:
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import pandas as pd

In [66]:
le_train_type = preprocessing.LabelEncoder()
le_train_type.fit(arrival_pd_df['train_type'])
# see the class
#list(le_train_type.classes_)

['EC', 'IC', 'IR', 'NJ', 'RE', 'S', 'SN']

In [67]:
arrival_pd_df['train_type_cat'] = le_train_type.transform(arrival_pd_df['train_type'])
# we can do the inverse
# arrival_pd_df['train_type'] = le_train_type.inverse_transform(arrival_pd_df['train_type_cat'])

array(['IR', 'IR', 'IC'], dtype=object)

In [79]:
arrival_pd_df

,trip_id,stop_id,schedule_dep,transport_type,train_type,weekday,next_sid,next_sched_arr,next_real_arrival,delay_arrival,arrival_interval,train_type_cat
0,85:11:13752:001,8502221,2018-01-28 02:12:00,Zug,SN,6,8502222,2018-01-28 02:16:00,2018-01-28 02:18:00,120,1,6
1,85:11:13752:001,8502229,2018-01-28 02:07:00,Zug,SN,6,8502221,2018-01-28 02:12:00,2018-01-28 02:13:00,60,1,6
2,85:11:13752:001,8502220,2018-01-28 02:06:00,Zug,SN,6,8502229,2018-01-28 02:07:00,2018-01-28 02:10:00,180,1,6
3,85:11:13752:001,8503001,2018-01-28 02:01:00,Zug,SN,6,8502220,2018-01-28 02:06:00,2018-01-28 02:08:00,120,1,6
4,85:11:13752:001,8503020,2018-01-28 01:59:00,Zug,SN,6,8503001,2018-01-28 02:01:00,2018-01-28 02:02:00,60,1,6
5,85:11:13752:001,8503000,2018-01-28 01:57:00,Zug,SN,6,8503020,2018-01-28 01:59:00,2018-01-28 01:59:00,0,1,6
6,85:11:13752:001,8503003,2018-01-28 01:52:00,Zug,SN,6,8503000,2018-01-28 01:55:00,2018-01-28 01:54:00,-60,1,6
7,85:11:13752:001,8502221,2018-01-27 02:12:00,Zug,SN,5,8502222,2018-01-27 02:16:00,2018-01-27 02:18:00,120,1,6
8,85:11:13752:001,8502229,2018-01-27 02:07:00,Zug,SN,5,8502221,2018-01-27 02:12:00,2018-01-27 02:13:00,60,1,6
9,85:11:13752:001,8502220,2018-01-27 02:06:00,Zug,SN,5,8502229,2018-01-27 02:07:00,2018-01-27 02:10:00,180,1,6


In [80]:
arrival_pd_df.weekday = arrival_pd_df.weekday.astype(int)
arrival_pd_df.arrival_interval = arrival_pd_df.arrival_interval.astype(int)
arrival_pd_df.arrival_interval = arrival_pd_df.arrival_interval.astype(int)

arrival_pd_df.dtypes

trip_id              object
stop_id               int64
schedule_dep         object
transport_type       object
train_type           object
weekday               int64
next_sid              int64
next_sched_arr       object
next_real_arrival    object
delay_arrival         int64
arrival_interval      int64
train_type_cat        int64
dtype: object

In [119]:
week_df = arrival_pd_df[arrival_pd_df.weekday == 0]
week_df.delay_arrival = week_df.delay_arrival/60
week_df

/opt/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,trip_id,stop_id,schedule_dep,transport_type,train_type,weekday,next_sid,next_sched_arr,next_real_arrival,delay_arrival,arrival_interval,train_type_cat
56,85:11:13752:001,8502221,2018-01-01 02:12:00,Zug,SN,0,8502222,2018-01-01 02:16:00,2018-01-01 02:30:00,14.0,1,6
57,85:11:13752:001,8502229,2018-01-01 02:07:00,Zug,SN,0,8502221,2018-01-01 02:12:00,2018-01-01 02:24:00,12.0,1,6
58,85:11:13752:001,8502220,2018-01-01 02:06:00,Zug,SN,0,8502229,2018-01-01 02:07:00,2018-01-01 02:20:00,13.0,1,6
59,85:11:13752:001,8503001,2018-01-01 02:01:00,Zug,SN,0,8502220,2018-01-01 02:06:00,2018-01-01 02:18:00,12.0,1,6
60,85:11:13752:001,8503020,2018-01-01 01:59:00,Zug,SN,0,8503001,2018-01-01 02:01:00,2018-01-01 02:13:00,12.0,1,6
61,85:11:13752:001,8503000,2018-01-01 01:57:00,Zug,SN,0,8503020,2018-01-01 01:59:00,2018-01-01 02:07:00,8.0,1,6
62,85:11:13752:001,8503003,2018-01-01 01:52:00,Zug,SN,0,8503000,2018-01-01 01:55:00,2018-01-01 02:01:00,6.0,1,6
77,85:11:18388:001,8503509,2018-01-29 23:38:00,Zug,S,0,8503512,2018-01-29 23:40:00,2018-01-29 23:41:00,1.0,1,5
78,85:11:18388:001,8503001,2018-01-29 23:36:00,Zug,S,0,8503509,2018-01-29 23:38:00,2018-01-29 23:38:00,0.0,1,5
79,85:11:18388:001,8503020,2018-01-29 23:31:00,Zug,S,0,8503001,2018-01-29 23:35:00,2018-01-29 23:34:00,-1.0,1,5


In [120]:
lin_regr = LinearRegression()
lin_regr.fit(week_df[['stop_id', 'next_sid', 'arrival_interval', 'train_type_cat']], week_df.delay_arrival)
pred = lin_regr.predict(week_df[['stop_id', 'next_sid', 'arrival_interval', 'train_type_cat']])
lin_regr.score(week_df[['stop_id', 'next_sid', 'arrival_interval', 'train_type_cat']],  week_df.delay_arrival)

0.030405173780714501

In [121]:
week_df['pred'] = pred
week_df

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,trip_id,stop_id,schedule_dep,transport_type,train_type,weekday,next_sid,next_sched_arr,next_real_arrival,delay_arrival,arrival_interval,train_type_cat,pred
56,85:11:13752:001,8502221,2018-01-01 02:12:00,Zug,SN,0,8502222,2018-01-01 02:16:00,2018-01-01 02:30:00,14.0,1,6,0.664332
57,85:11:13752:001,8502229,2018-01-01 02:07:00,Zug,SN,0,8502221,2018-01-01 02:12:00,2018-01-01 02:24:00,12.0,1,6,0.662825
58,85:11:13752:001,8502220,2018-01-01 02:06:00,Zug,SN,0,8502229,2018-01-01 02:07:00,2018-01-01 02:20:00,13.0,1,6,0.666304
59,85:11:13752:001,8503001,2018-01-01 02:01:00,Zug,SN,0,8502220,2018-01-01 02:06:00,2018-01-01 02:18:00,12.0,1,6,0.542158
60,85:11:13752:001,8503020,2018-01-01 01:59:00,Zug,SN,0,8503001,2018-01-01 02:01:00,2018-01-01 02:13:00,12.0,1,6,0.741728
61,85:11:13752:001,8503000,2018-01-01 01:57:00,Zug,SN,0,8503020,2018-01-01 01:59:00,2018-01-01 02:07:00,8.0,1,6,0.749774
62,85:11:13752:001,8503003,2018-01-01 01:52:00,Zug,SN,0,8503000,2018-01-01 01:55:00,2018-01-01 02:01:00,6.0,1,6,0.744120
77,85:11:18388:001,8503509,2018-01-29 23:38:00,Zug,S,0,8503512,2018-01-29 23:40:00,2018-01-29 23:41:00,1.0,1,5,0.516271
78,85:11:18388:001,8503001,2018-01-29 23:36:00,Zug,S,0,8503509,2018-01-29 23:38:00,2018-01-29 23:38:00,0.0,1,5,0.594724
79,85:11:18388:001,8503020,2018-01-29 23:31:00,Zug,S,0,8503001,2018-01-29 23:35:00,2018-01-29 23:34:00,-1.0,1,5,0.460024


In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

trip_Indexer = StringIndexer(inputCol="trip_id", outputCol="trip_id_index")
trip_model = trip_Indexer.fit(arrival_df)

arrival_df = trip_model.transform(arrival_df)

In [ ]:
arrival_df = arrival_df.drop('trip_id', 'transport_type')

In [ ]:
arrival_df = arrival_df.withColumn('weekday', arrival_df['weekday'].cast('int'))
arrival_df = arrival_df.withColumn('arrival_interval', arrival_df['arrival_interval'].cast('int'))

In [ ]:
arrival_df = arrival_df.withColumn('delay_arrival', arrival_df['delay_arrival'] / 60)

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler


vec_ass = VectorAssembler(inputCols=['stop_id','weekday','arrival_interval','trip_id_index','transport_type_index'], outputCol="features")

#arrival_df = vec_ass.transform(arrival_df)
lr = LinearRegression(maxIter=5, regParam=0.0, solver="normal", 
                      labelCol='delay_arrival')
model = lr.fit(arrival_df)

test = model.transform(arrival_df)

In [ ]:
test = test.withColumn('evaluation', test['delay_arrival'] - test['prediction'])

In [ ]:
test.describe('evaluation').show()

In [ ]:
test.describe('delay_arrival').show()

In [ ]:
test.describe('prediction').show()

In [ ]:
test_df = arrival_df.limit(50).toPandas()
test_df